# Correction Grids

Correction grids can be thought of as lookup tables that provide corrected Petrosian derived values. 
These grids are typically generated through simulations or theoretical models to offer 
the best correction for Petrosian profiles. This section discusses the concept of correction grids: how they're generated,
when and why they should be used, and how they enhance accuracy.

In [ ]:
# Hidden cell

%matplotlib inline

## Necessity and Implications

**Are Correction Grids Always Needed?**

The short answer is no, but they are very useful in some cases. 
Petrosian corrections offer more precise measurements of galaxies. 
The necessity hinges on your data quality, research goals, and desired precision. 
Here are key considerations on why the defualt settings are usually good enough:

1. **Standard Practices**: Numerous studies, such as those from the Sloan Digital Sky Survey (SDSS), 
   rely on default settings. The parameters $\eta = 0.2$ and $\epsilon = 2$ are widely accepted because
   they typically produce reliable results across many scenarios.

2. **Noise Constraints**: If the fainter parts of galaxies in your images are submerged below the noise level, 
   detailed corrections might not significantly impact the outcome since that portion of the galaxy is not meaningfully
   measurable. Here, the additional steps and computational demands of correction grids might be unwarranted.

3. **Practical Implications**: While correction grids can produce improved results, 
   they require optimal background subtraction. Not all datasets might meet this standard.
 
Correction grids promise enhanced precision but aren't always essential. 
For a majority of cases, default settings suffice.

**When are Correction Grids Needed?**

While the default settings serve most general purposes, there are specific scenarios where 
correction grids are beneficial:

1. **Galaxy Size Estimation**: If your research aims to measure the size of a galaxy which suffers due to 
   poor signal-to-noise, you can estimate its total flux radius based on the corrected epsilon. Remember, 
   this isn't about recovering the total flux which includes regions submerged under noise, but 
   about the estimating the full extent of the galaxy based on the measurable components of the light profile. 

2. **Estimating Sérsic Index**: Correction grids can be used to estimate the Sérsic index of single component galaxies.

3. **High-Precision Studies**: For projects that require a granular study of galaxy structures, 
   or when analyzing galaxies with distinct features that might get overlooked with standard parameters, 
   correction grids provide the extra precision that may be needed.

4. **Superior Data Quality**: If you have high-quality images with excellent background subtraction 
   and reasonable signal-to-noise ratios, the total flux can be estimated to higher accuracy. That is,
   high quality data allows you to measure the flux up to the corrected total flux radius. 

In essence, when the goal shifts from general measurements to intricate, high-resolution studies of galactic structures and sizes, correction grids become a useful tool.



## Generating a Correction Grid

Correction grids are essentially lookup tables that map certain observational properties of a galaxy 
(like Petrosian radius, uncorrected half light radius, and concentration indices) to more intrinsic 
properties (like Sérsic index and the corrected epsilon value). In this section, we'll walk through the steps of creating a correction grid. 

To start with `PetroFit`, simply import it as follows:


In [ ]:
import petrofit as pf

### Load PSF (Optional)

We load an HST `F105W` PSF and normalize it to sum to 1:

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['font.size'] = 12

In [ ]:
from astropy.io import fits

# Load PSF image (2D array)
PSF = fits.getdata('data/f105w_psf.fits.gz')

# Normalize PSF 
PSF = PSF / PSF.sum()

# Note that the PSF shape is odd on all sides
print("PSF Shape = {}".format(PSF.shape))

# Plot PSF and use vmax and vmin to show difraction spikes
plt.imshow(PSF, vmin=0, vmax=5e-4)
plt.show()



### Define Sampling Points 

The correction grid is generated along with a set of half-light radii and Sersic indices. The generator loops through the Sersic indices for each half-light radius in the radius list. For this documentation, we define a small set of values, with the Sersic indices of a Gaussian and de Vaucouleurs’ profile.


In [ ]:
import numpy as np 

r_eff_list = np.array([7, 10, 15])
n_list = np.array([0.5, 1., 4.])

### Grid Simulation

We do a simple API call to generate the correction grid. We provide a PSF and oversampling rule as well. Oversampling becomes important for small half-light radii since the model needs to be sampled well in the center. 

The `generate_petrosian_sersic_correction` grid follows the following steps to generate the correction grid:

- Computes the total flux (`total_flux`) of an ideal Sersic profile with the sampling points using the ` petrofit.modeling.models.sersic_enclosed` function and setting the radius to `np.inf`.
- Computes the radius equal to `total_flux * 0.99`.
- Makes a PSF convolved Sersic Model image and measures the photometry. 
- Measures the uncorrected Petrosian radius. 
- Computes the correct `epsilon` value as ` corrected_epsilon = r(total_flux) / r_petrosian`.
- It also saves other values, such as the uncorrected `C2080` which can be used to mapto the Sersic index.


In [ ]:
output_file_name='temp/example_correction_gid.ecsv'

petrosian_grid = pf.generate_petrosian_sersic_correction(
    output_file_name=output_file_name, # Output file name
    psf=PSF, # PSF (optional)
    r_eff_list=r_eff_list, # List of r_e to sample
    n_list=n_list, # List of n to sample
    oversample=4, # Oversample factor or region, see fitting docs
    out_format='ascii.ecsv', # Output format, should match file name
    overwrite=True, # Overwrite output
    ipython_widget=True, # Progress bar
    n_cpu=None, # int value for number of mp threads
    plot=False, # Plot each step, can not be True if n_cpu > 1

)

In [ ]:
petrosian_grid

## PetrosianCorrection


The `PetrosianCorrection` class is designed to compute corrections for Petrosian based on 
default Petrosian measurements. 

### Loading and Navigating

1. **Grid Input**: To initialize the `PetrosianCorrection` class, you'll need a correction grid. 
   This grid is typically an Astropy Table that has been generated with the function 
   `petrofit.correction.generate_petrosian_sersic_correction`.

In [ ]:
# Input generated grid table
pc = pf.PetrosianCorrection(petrosian_grid)

2. **Reading and Writing Grids**: 
    - Use the `PetrosianCorrection.read(file_path)` class method to read a correction grid from a file.
    - Similarly, to save your correction grid to a file, utilize the `write(grid_file, file_format=None)` method.


In [ ]:
# Read F105W grid:
pc = pf.PetrosianCorrection.read(output_file_name)


3. **Working with Grid Data**:
    - The `grid_keys` property will provide you with the column names of your grid.
    - Use `unique_grid_values(key)` to obtain unique values from a specific column.
    - The `filter_grid(key, value)` method helps filter the grid based on a specific key and value.


In [ ]:
# Read ideal Sersic grid:
pc = pf.PetrosianCorrection.read('data/no_psf_corr.ecsv')

In [ ]:
print(pc.grid_keys)

In [ ]:
# List all Sersic indices available 
pc.unique_grid_values('n')

In [ ]:
# Provide rows for a grid key's value
# For example all rows with n=1
pc.filter_grid('n', 1.)

In [ ]:
# Plotting relations 
pc_c_c2080_list = np.sort(pc.unique_grid_values('c_c2080')) # Get corrected c_c2080 
approx_n_values = pf.PetroApprox.c2080_to_n(pc_c_c2080_list) # Get n from PetroApprox for comparison

# Plot
plt.scatter( pc.grid['c_c2080'], pc.grid['n'], color='black')
plt.plot(pc_c_c2080_list, approx_n_values, color='red')

plt.xlabel('Corrected $C_{2080}$')
plt.ylabel('n')
plt.show()

### Correcting a Petrosian Profile

In the [Photometry Chapter](./photometry.ipynb#Photometry) we constructed a curve of growth for the football shaped galaxy:


In [ ]:
import numpy as np
from astropy.table import Table
phot_table = Table.read('data/abell_2744_galaxy_f105w_photometry.ecsv') # Read table 

# Load data
r_list = np.array(phot_table['r_list'])
flux_arr = np.array(phot_table['flux_arr'])
area_arr = np.array(phot_table['area_arr'])
error_arr = np.array(phot_table['error_arr'])


# Make Petrosian profile
p = pf.Petrosian(r_list, area_arr, flux_arr, flux_err=error_arr)

p.plot()
plt.show()

1. **Correct Profile**:
    - Simply use the `correct(p)` function to correct the profile. 
    - Use `plot_correction` function to see the closest value.
    - The parameters used for correction are derived directly from the grid. Specifically:
        - `'p02'`: Represents the Petrosian radius $P_{02} = R(\eta_{0.2})$.
        - `'u_r_50'`: Stands for the uncorrected half-light radius.
        - `'u_c2080'`: Corresponds to the uncorrected concentration index.


In [ ]:
pc = pf.PetrosianCorrection.read('data/f105w_psf_corr.ecsv')

# Plot grid and uncorrected profile on grid (p)
pc.plot_correction(p)
plt.show()

# Pass uncorrected p to the correct function
p_corrected = pc.correct(p)

# Plot corrected profile
p_corrected.plot()
plt.show()

2. **Estimating Parameters**:
    - The method `estimate_n(p)` will give an estimated Sérsic index `n` based on the half-light radius and `c2080` computed using the default epsilon value.
    - The method `estimate_epsilon(p)` will provide a corrected epsilon value given the half-light radius and `c2080` computed with the default epsilon.



In [ ]:
n = pc.estimate_n(p)
n 

In [ ]:
corr_epsilon = pc.estimate_epsilon(p)
corr_epsilon